<h1 align="center">Design an application for public health - Project 3</h1>
<h2 align="center">Cleaning notebook</h2>
<h3 align="center">Data Scientist course - OpenClassrooms</h3>

<h1><center>Hola mundo</center><h1>
    
    dasdasdasd

<div class="alert alert-block alert-info">
    This <b>notebook</b> is optimized to <b>be used</b> with:
    <ul style="list-style-type: square;">
    <li>JupyterLab 3.0.14</li>
    <li>JupyterLab Theme: JupyterLab Dark</li>
    <li>Extension: @jupyter-widgets/jupyterlab-manager</li>
</ul>
</div>

## <center>Your centered level h2 title</center>

<h3 align="center">This is a centered header</h3>